In [7]:
!pip install -U /kaggle/input/bitsandbytes-0-42-0-py3-none-any-whl/bitsandbytes-0.42.0-py3-none-any.whl -qq

# LLM inference

When you develop your prompts, I advice you make inference from a much faster device.

However, you do need to make sure that the model outputs will be the same.

In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig

MODEL_PATH = "/kaggle/input/mixtral/pytorch/8x7b-instruct-v0.1-hf/1"

quantization_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

config = AutoConfig.from_pretrained(MODEL_PATH)
config.gradient_checkpointing = True
config.do_sample = True

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map = "auto",
    trust_remote_code = True,
    quantization_config=quantization_config,
    config=config
)

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

In [9]:
import textwrap
import transformers


def make_inference(instruction):
    instruction_format = "[INST]{instruction}[/INST]"
    text = instruction_format.format(instruction=instruction)
    inputs = tokenizer(text, return_tensors="pt").to('cuda')

    outputs = model.generate(**inputs, max_new_tokens=2000)
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return output_text

In [10]:
instruction = """
Write Python code to solve this problem in a brute force manner. You will only print the final result.

Find the largest two digit prime number which is also a prime number when the digits are reversed.

Reminder: You will write Python code. You do not care about the efficiency of the solution, you will only care about the correctness.
""".strip()

In [11]:
output_text = make_inference(instruction)

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1339: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
2024-06-13 11:24:35.556110: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-13 11:24:35.556199: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-13 11:24:35.687020: E external/local_xla/xla/stream_ex

In [12]:
print(output_text)

[INST]Write Python code to solve this problem in a brute force manner. You will only print the final result.

Find the largest two digit prime number which is also a prime number when the digits are reversed.

Reminder: You will write Python code. You do not care about the efficiency of the solution, you will only care about the correctness.[/INST] Here is a brute force solution to find the largest two digit prime number which is also a prime number when the digits are reversed:

```python
def is\_prime(n):
for i in range(2, int(n**0.5) + 1):
if n % i == 0:
return False
return True

large_prime = -1
for i in range(90, 10, -1):
if is\_prime(i) and is\_prime(int(str(i)[::-1])):
large\_prime = i
break

print(large\_prime)
```

This solution uses a helper function `is_prime` to check if a number is prime. The solution iterates from 90 to 10 in steps of -1, checks if the number and its reverse are both prime, and if so, stores the number in the `large_prime` variable and breaks the loop. Fi

# Code execution

In [13]:
import re
def extract_code(reply):
    pattern = r"```python([\s\S]*?)```"
    matches = re.findall(pattern, reply)
    code = "\n\n".join(matches)
    if code:
        return code
    pattern = r"```([\s\S]*?)```"
    matches = re.findall(pattern, reply)
    code = "\n\n".join(matches)
    return code

In [14]:
import sys
import io
import contextlib
import signal

def run_python_code(code, time_limit_seconds=20):
    @contextlib.contextmanager
    def time_limit(seconds):
        def signal_handler(signum, frame):
            raise TimeoutError("Timed out!")
        signal.signal(signal.SIGALRM, signal_handler)
        signal.alarm(seconds)
        try:
            yield
        finally:
            signal.alarm(0)

    old_stdout = sys.stdout
    redirected_output = sys.stdout = io.StringIO()

    try:
        with time_limit(time_limit_seconds):
            exec(code)
    except TimeoutError as e:
        print("Code execution timed out")
    except Exception as e:
        print(f"Error: {str(e)}")
    finally:
        sys.stdout = old_stdout

    return redirected_output.getvalue()

In [15]:
execution_output = run_python_code("""
print(1+3)
""", time_limit_seconds=5)

print(execution_output)

4



In [16]:
execution_output = run_python_code("""
import time
for i in range(10):
    print(i)
    time.sleep(1)
""", time_limit_seconds=5)

print(execution_output)

0
1
2
3
4
Code execution timed out



# Produce answers

In [17]:
import pandas as pd
df = pd.read_csv("/kaggle/input/ai-mathematical-olympiad-prize/test.csv")

In [18]:
from collections import defaultdict

instruction_template = """
Write Python code to solve this problem in a brute force manner. You will only print the final result.

{problem}

Reminder:
- You will write Python code.
- You will only attempt to solve the problem with brute force.
- You will only print the final result.
""".strip()

answers = []

for problem in df["problem"]:
    print(problem)
    responses = defaultdict(int)
    
    for _ in range(3):
        instruction = instruction_template.format(problem=problem)
        output_text = make_inference(instruction)
        print(f"{output_text=}")
        code = extract_code(output_text)
        print(f"{code=}")
        execution_output = run_python_code(code)
        print(f"{execution_output=}")
        try:
            result = int(execution_output)
            responses[result] += 1
        except ValueError:
            continue
        
    if len(responses) == 0:
        answer = 1
        answers.append(answer)  # random guess
    else:
        answer = sorted([(v,k) for k,v in responses.items()])[-1][-1]
        answers.append(answer%1000)  # most common guess

df["answer"] = answers

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


What is $1-1$?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


output_text='[INST]Write Python code to solve this problem in a brute force manner. You will only print the final result.\n\nWhat is $1-1$?\n\nReminder:\n- You will write Python code.\n- You will only attempt to solve the problem with brute force.\n- You will only print the final result.[/INST] Here is a brute force way to solve the problem of finding the value of 1 - 1 in Python:\n\n```python\n# Brute force method to find the value of 1 - 1\n\nresult = 1 - 1\n\nprint("The result is: ", result)\n```\n\nOutput:\n\n```\nThe result is:  0\n```\n\nIn this code, we define the result as the expression `1 - 1`, and then we print the result. The brute force method is used because there is only one simple operation to perform.'
code='\n# Brute force method to find the value of 1 - 1\n\nresult = 1 - 1\n\nprint("The result is: ", result)\n'
execution_output='The result is:  0\n'


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


output_text='[INST]Write Python code to solve this problem in a brute force manner. You will only print the final result.\n\nWhat is $1-1$?\n\nReminder:\n- You will write Python code.\n- You will only attempt to solve the problem with brute force.\n- You will only print the final result.[/INST] Here is a brute force solution using Python:\n\n```python\n# Brute force solution\n\n# Initialize the answer variable\nanswer = 1 - 1\n\n# Print the final result\nprint(answer)\n```\n\nThis code initializes the answer variable to the result of subtracting 1 from 1, then prints the final result. While this is a brute force solution, it is also a very simple one that only requires a single line of code to solve the problem. However, it is worth noting that in more complex problems, a brute force approach may not be the most efficient or practical solution.'
code='\n# Brute force solution\n\n# Initialize the answer variable\nanswer = 1 - 1\n\n# Print the final result\nprint(answer)\n'
execution_out

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


output_text="[INST]Write Python code to solve this problem in a brute force manner. You will only print the final result.\n\nWhat is $1-1$?\n\nReminder:\n- You will write Python code.\n- You will only attempt to solve the problem with brute force.\n- You will only print the final result.[/INST] Here is a brute force solution using Python:\n\n```\n# Brute force solution to find the value of 1 - 1\n\n# Initialize the result variable\nresult = 0\n\n# Try subtracting 1 from 1\nresult = 1 - 1\n\n# Print the final result\nprint(result)\n```\n\nThis code initializes a variable `result` to 0, then assigns it the value of `1 - 1`. Finally, it prints the value of `result`.\n\nOf course, this is a very simple problem and doesn't necessarily require a brute force solution, but this code will correctly solve the problem using brute force."
code='\n# Brute force solution to find the value of 1 - 1\n\n# Initialize the result variable\nresult = 0\n\n# Try subtracting 1 from 1\nresult = 1 - 1\n\n# Prin

KeyboardInterrupt: 

In [19]:
print(code)


# Brute force solution to find the value of 1 - 1

# Initialize the result variable
result = 0

# Try subtracting 1 from 1
result = 1 - 1

# Print the final result
print(result)



In [20]:
df

,row_id,id,problem
0,0,000aaa,What is $1-1$?
1,1,111bbb,What is $0\times10$?
2,2,222ccc,Solve $4+x=4$ for $x$.


In [ ]:
df[["id", "answer"]].to_csv("submission.csv", index=False)

In [ ]:
!head /kaggle/input/ai-mathematical-olympiad-prize/sample_submission.csv

In [ ]:
!head submission.csv